In [ ]:
# importing all the required libraries

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
#using mobilenet it supports various input size, here i am using 128x128x3
mobile_net=tf.keras.applications.MobileNet(input_shape=(128,128,3))

In [ ]:
# importing model and Layers from tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,GlobalAveragePooling2D,Dropout

In [ ]:
#summary
mobile_net.summary()

In [ ]:
# from global_average_pooling2d which is the flattening layer onwards are the dense layer architecture

In [ ]:
# we are not including the Dense layer architecture in our base_model
base_model=tf.keras.applications.MobileNet(input_shape=(128,128,3),include_top=False) #192,160,148,224 input shapes
# for mobilenet
base_model.summary()

In [ ]:
# from the base modelsummary it is clear that flatten and dense layers are removed
# we need to freeze trainable weights

In [ ]:
base_model.trainable

In [ ]:
base_model.trainable=False # deliberately making it False

In [ ]:
base_model.summary()

In [ ]:
# all weights are now freezed as Trainable params are zero

In [ ]:
# now designing dense layers

In [ ]:
#our model architecture
transfer_model=Sequential()
transfer_model.add(base_model)# base model is added
transfer_model.add(Flatten()) # increases weights and time consuming
#transfer_model.add(GlobalAveragePooling2D())
transfer_model.add(Dense(units=512,activation='relu'))
transfer_model.add(Dropout(.2))
transfer_model.add(Dense(units=64,activation='relu'))
transfer_model.add(Dropout(.1))
transfer_model.add(Dense(units=5,activation='softmax'))


In [ ]:
transfer_model.summary()

In [ ]:
# training and testing data are generated 
#in train data augmentation is also used

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
                  rotation_range=45,
                  width_shift_range=0.2,
                  height_shift_range=0.2,
                  shear_range=0.2, # shrink
                  zoom_range=0.2,
                  horizontal_flip=True,# becareful with flip dont flip numbers
                  fill_mode='reflect'
                   
                  )
test_datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

In [ ]:
# batch_size=128

In [ ]:
train_set=train_datagen.flow_from_directory("C:\\Users\\warda\\flower_photos\\Training",
                                 target_size=(128,128),
                                 batch_size=128,
                                 class_mode='categorical') # for binary give class_mode=binary

test_set=test_datagen.flow_from_directory("C:\\Users\\warda\\flower_photos\\Testing",
                                 target_size=(128,128),
                                 batch_size=128,
                                 class_mode='categorical')# images are taken in random way not sequential

In [ ]:
#compiling the model
#using Adam as optimizer

In [ ]:
transfer_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#callbacks and earlystopping are used to store model weights of best model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

keras_callback=[EarlyStopping(monitor='val_loss',mode='min',patience=5,min_delta=.01),
ModelCheckpoint('C:\\Users\\warda\\Flowerbest_transfer',monitor='val_loss',save_best_only=True)] 

In [ ]:
#fit the model

In [ ]:
transfer_model.fit(train_set,
                      steps_per_epoch=2736//128, # how many times in one epoch weights need to be updated.
                      epochs=5, # try 20
                      validation_data=test_set,
                      validation_steps=934//128,
                  callbacks=keras_callback)

In [ ]:
transfer_model.evaluate(test_set)

In [ ]:
#testing the model

In [ ]:
test_img=image.load_img('C:\\Users\\warda\\flower_photos\\example\\11.jpg',target_size=(128,128))


test_img1=image.img_to_array(test_img)
test_img1=tf.keras.applications.mobilenet.preprocess_input(test_img1)
test_img2=np.expand_dims(test_img1,axis=0)
# just one image for testing
ypred=transfer_model.predict(test_img2)
print(train_set.class_indices)
print("The test image class is",ypred.argmax())
class_name=train_set.class_indices
pos=np.array(list(class_name.values()))==ypred.argmax()
name=np.array(list(class_name.keys()))
print("The predicted class name is:")
print(name[pos][0])
test_img

In [ ]:
#forming confusion matrix

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

test_set=test_datagen.flow_from_directory('C:\\Users\\warda\\flower_photos\\Testing',
                                 target_size=(128,128),
                                 shuffle=False,
                                 batch_size=1,     
                                 class_mode='categorical') # binary for covid-noncovid(2 classes)

filenames = test_set.filenames
nb_samples = len(filenames)

predict = transfer_model.predict_generator(test_set,steps = nb_samples)

In [ ]:
ypred=predict.argmax(axis=1)
ypred

In [ ]:
yact=test_set.classes
yact

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sb
plt.figure(figsize=(10,6))
sb.heatmap(confusion_matrix(yact,ypred),annot=True,fmt='.0f')

In [ ]:
# with respect to tulip my model is not doing well

In [ ]:
# remedy to change global avg pool to flatten and check it
# resize image

In [ ]:
# tulips predicted as rose (29) class 4 has more wrong predictions

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(yact,ypred))

In [ ]:
#  i want to know 27 wrong classifications as images 
import pandas as pd

In [ ]:
yact1=pd.DataFrame(yact)
ypred1=pd.DataFrame(ypred)

y=pd.concat([yact1,ypred1],axis=1)

In [ ]:
y.columns=['yact','ypred']
y

In [ ]:
y[(y['yact']==4)&(y['ypred']==2)]


# Transfer Learning to Fine Tuning

In [ ]:
base_model=tf.keras.applications.MobileNet(input_shape=(128,128,3),include_top=False) # here top is Dense layer

In [ ]:
base_model.summary() # load only convolutional layers

In [ ]:
base_model.layers  # batch normalization is to scale data so data will converge

In [ ]:
#no of layers
len(base_model.layers) 

In [ ]:
base_model.layers[71:] 

In [ ]:
base_model.trainable

In [ ]:
for layer in base_model.layers[:71]:
    layer.trainable=False
    
    # freezing the first 70 layers


In [ ]:
#printing names if layers
for layer in base_model.layers:
    print(layer.name,layer.trainable)

In [ ]:
# model architecture
transfer_model_t=Sequential()
transfer_model_t.add(base_model)
transfer_model_t.add(GlobalAveragePooling2D())
transfer_model_t.add(Dense(units=512,activation='relu'))
transfer_model_t.add(Dropout(.2))
transfer_model_t.add(Dense(units=64,activation='relu'))
transfer_model_t.add(Dropout(.1))
transfer_model_t.add(Dense(units=5,activation='softmax'))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
                  rotation_range=45,
                  width_shift_range=0.2,
                  height_shift_range=0.2,
                  shear_range=0.2, # shrink
                  zoom_range=0.2,
                  horizontal_flip=True,# becareful with flip dont flip numbers
                  fill_mode='reflect'
                   
                  )
test_datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

In [ ]:
train_set=train_datagen.flow_from_directory("C:\\Users\\warda\\flower_photos\\Training",
                                 target_size=(128,128),
                                 batch_size=128,
                                 class_mode='categorical') # for binary give class_mode=binary

test_set=test_datagen.flow_from_directory("C:\\Users\\warda\\flower_photos\\Testing",
                                 target_size=(128,128),
                                 batch_size=128,
                                 class_mode='categorical')# images are taken in random way not sequential

In [ ]:
#using Adam as optimizer
ada=tf.keras.optimizers.Adam

In [ ]:
#compiling the model
transfer_model_t.compile(optimizer=ada(learning_rate=.0001),loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

keras_callback=[EarlyStopping(monitor='val_loss',mode='min',patience=5,min_delta=.01),
ModelCheckpoint('C:\\Users\\warda\\Flowerbest_transfertt',monitor='val_loss',save_best_only=True)] 

In [ ]:
transfer_model_t.fit(train_set,
                      steps_per_epoch=2736//128, 
                      epochs=5, # try 20
                      validation_data=test_set,
                      validation_steps=934//128,
                  callbacks=keras_callback)

In [ ]:
#evaluating the model

In [ ]:
transfer_model_t.evaluate(test_set)

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

test_set=test_datagen.flow_from_directory('C:\\Users\\warda\\flower_photos\\Testing',
                                 target_size=(128,128),
                                 shuffle=False,
                                 batch_size=1,     
                                 class_mode='categorical') # binary for covid-noncovid(2 classes)

filenames = test_set.filenames
nb_samples = len(filenames)

predict = transfer_model_t.predict_generator(test_set,steps = nb_samples)

In [ ]:
ypred=predict.argmax(axis=1)
yact=test_set.classes
from sklearn.metrics import confusion_matrix
import seaborn as sb
plt.figure(figsize=(10,6))
sb.heatmap(confusion_matrix(yact,ypred),annot=True,fmt='.0f')